### Importação das bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

### Carregamento dos arquivos

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

### Criação do DataFrame de saída

In [ ]:
resp = pd.DataFrame(columns=["NU_INSCRICAO", "NU_NOTA_MT"])
resp["NU_INSCRICAO"] = test["NU_INSCRICAO"]

### Seleção dos atributos mais "importantes"

In [ ]:
novos = [ 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_COMP3', 'NU_NOTA_REDACAO', 'NU_NOTA_LC', 'NU_NOTA_CH', 'NU_NOTA_CN',]
attributes_1 = ["NU_IDADE", "TP_ST_CONCLUSAO", "TP_ANO_CONCLUIU", "TP_ESCOLA", "TP_ENSINO", "NU_NOTA_MT"] + novos
attributes_2 = ["NU_IDADE", "TP_ST_CONCLUSAO", "TP_ANO_CONCLUIU", "TP_ESCOLA", "TP_ENSINO"] + novos

# Seleção das colunas
train = train[attributes_1]
test = test[attributes_2]

### Preenchimento das colunas com atributos ausentes

In [ ]:
train.loc[train["TP_ENSINO"].isna(), "TP_ENSINO"] = 1
train.loc[train["TP_ENSINO"].isna(), "TP_ENSINO"] = 1
train.loc[train["NU_NOTA_MT"].isna(), "NU_NOTA_MT"] = 0
train.loc[train["NU_NOTA_COMP1"].isna(), "NU_NOTA_COMP1"] = 0
train.loc[train["NU_NOTA_COMP2"].isna(), "NU_NOTA_COMP2"] = 0
train.loc[train["NU_NOTA_COMP3"].isna(), "NU_NOTA_COMP3"] = 0
train.loc[train["NU_NOTA_COMP4"].isna(), "NU_NOTA_COMP4"] = 0
train.loc[train["NU_NOTA_COMP5"].isna(), "NU_NOTA_COMP5"] = 0

train.loc[train["NU_NOTA_REDACAO"].isna(), "NU_NOTA_REDACAO"] = 0
train.loc[train["NU_NOTA_LC"].isna(), "NU_NOTA_LC"] = 0
train.loc[train["NU_NOTA_CH"].isna(), "NU_NOTA_CH"] = 0
train.loc[train["NU_NOTA_CN"].isna(), "NU_NOTA_CN"] = 0


In [ ]:
train.isnull().sum()

NU_IDADE           0
TP_ST_CONCLUSAO    0
TP_ANO_CONCLUIU    0
TP_ESCOLA          0
TP_ENSINO          0
NU_NOTA_MT         0
NU_NOTA_COMP1      0
NU_NOTA_COMP2      0
NU_NOTA_COMP4      0
NU_NOTA_COMP5      0
NU_NOTA_COMP3      0
NU_NOTA_REDACAO    0
NU_NOTA_LC         0
NU_NOTA_CH         0
NU_NOTA_CN         0
dtype: int64

### Separação da base de treino


In [ ]:
label = train['NU_NOTA_MT']
# Exclusão da coluna alvo do conjunto de treino
del train["NU_NOTA_MT"]

X_train, X_test, y_train, y_test = train_test_split(train, label, test_size=0.2, random_state = 0)

### Execução dos algoritmos no conjunto de treino

In [ ]:
algorithms = [LinearRegression(), XGBRegressor(objective='reg:squarederror', n_estimators=200, learning_rate=0.1)]

best_acc = 0
for i, alg in enumerate(algorithms):
    alg.fit(X_train, y_train)
    acc = alg.score(X_train, y_train)

    if acc > best_acc:
        best_acc = acc
        best_alg = i
    
    print(alg.__class__.__name__ + ' ' + str(acc*100))
    # pred = alg.predict()

LinearRegression 90.28762129645939
XGBRegressor 93.16949391520394


### Execução do melhor algoritmo no conjunto de teste


In [ ]:
alg = algorithms[best_alg]
alg.fit(train, label)
pred = alg.predict(test)

### Escrita do arquivo de saída

In [ ]:
resp["NU_NOTA_MT"] = pred
resp.to_csv("Notas.csv", index=False)